In [3]:
import numpy as np
from py_vollib.black_scholes.implied_volatility import implied_volatility as iv

In [4]:
BUSINESS_DAYS_IN_A_YEAR = 256
def date_out_of_year(day : int):
    return day / BUSINESS_DAYS_IN_A_YEAR

def volatility_call(time: int, stock_price: float, strike_price: float,
                    interest_rate: float, call_price: float):
    imp_v = iv(call_price, stock_price, strike_price, date_out_of_year(time), interest_rate, 'c')
    return imp_v

def volatility_put(time: int, stock_price: float, strike_price: float,
                    interest_rate: float, put_price: float):
    imp_v = iv(put_price, stock_price, strike_price, date_out_of_year(time), interest_rate, 'p')
    return imp_v

In [5]:
class ProductDetails:
    def __init__(self, type: str, time: int, stock_price: float, strike_price: float,
                 interest_rate: float, option_price: float, input_amount:int, round_lot:int=100,
                 dividend: float=0):

        imp_volatility = 0
        if type == "call":
            imp_volatility = volatility_call(time, stock_price, strike_price, interest_rate, option_price)
        elif type == "put":
            imp_volatility = volatility_put(time, stock_price, strike_price, interest_rate, option_price)
        else:
            raise Exception


        self.delta_1 = (np.log(stock_price / strike_price) +
                       (interest_rate - dividend + 0.5 * (imp_volatility ** 2)) * time) / \
                       (imp_volatility * np.sqrt(time))
        self.delta_2 = self.delta_1 - imp_volatility * np.sqrt(time)

        self.call_price = stock_price * np.exp(- dividend * time) * \
                          np.random.normal(self.delta_1) - \
                          strike_price * np.exp(- interest_rate * time) * \
                          np.random.normal(self.delta_2)

        self.put_price = strike_price * np.exp(- interest_rate * time) * \
                         np.random.normal(-self.delta_2) - \
                         stock_price * np.exp(- dividend * time) * \
                         np.random.normal(-self.delta_1)

        # delta amount
        delta_call = np.random.normal(self.delta_1)
        delta_put = np.random.normal(self.delta_2)

        self.delta_amount = (input_amount / round_lot) * delta_put / delta_call

In [6]:
# test 1
time = 0.0820313
stock_price = 5360
strike_price = 5360
call_price = 170.39716
interest_rate = 0.0218
input_amount = 2000
testProduct = ProductDetails("call", time, stock_price, strike_price, interest_rate, call_price, input_amount)
